In [1]:
import sys

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *

Lambda = Abstraction
App = Application

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [2]:
rs = RIStrategy()


def test_procedure(term, reduction_strategy=rs, print_steps=True,
                   steps_lim=60):
    term = term._update_bound_vars()
    n_steps = 0
    is_term_normalized = True
    if print_steps:
        print(f"{n_steps}: {term.funky_str()}")
    while term.redexes:
        term = term._beta_conversion(reduction_strategy)
        term = term._update_bound_vars()
        n_steps += 1
        if print_steps:
            print(f"{n_steps}: {term.funky_str()}")
        if n_steps > steps_lim:
            is_term_normalized = False
            break

    print(f"\n\nreduction steps: {n_steps}")
    print(f"norm term:       {term.funky_str()}" if is_term_normalized else f"NOT norm term: {term.funky_str()}")

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [3]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [4]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
        Lambda(x, Lambda(y, y_)),
        App(
            Lambda(x, App(x_, x_)),
            Lambda(x, App(x_, x_))
        )
    )


test_procedure(term_test_1())

0: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
1: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
2: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
3: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
4: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
5: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
6: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
7: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
8: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
9: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
10: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
11: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
12: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
13: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
14: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
15: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
16: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
17: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
18: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
19: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
20: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
21: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
22: ((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))
23: ((λa.(λb.b)) ((λx.(x x)) (λy.(y y))))
24

# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [5]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

0: ((((λx.(λy.(λa.((x y) a)))) (λb.(λc.b))) (λd.(λe.(d (d (d e)))))) (λj.(λi.(j i))))
1: (((λx.(λy.(((λa.(λb.a)) x) y))) (λc.(λd.(c (c (c d)))))) (λe.(λj.(e j))))
2: (((λx.(λy.((λa.x) y))) (λb.(λc.(b (b (b c)))))) (λd.(λe.(d e))))
3: (((λy.(λa.y)) (λb.(λc.(b (b (b c)))))) (λd.(λx.(d x))))
4: ((λx.(λy.(λa.(y (y (y a)))))) (λb.(λc.(b c))))
5: (λx.(λy.(x (x (x y)))))


reduction steps: 5
norm term:       (λx.(λy.(x (x (x y)))))


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [6]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

0: ((((λx.(λy.(λa.((x y) a)))) (λb.(λc.c))) (λd.(λe.e))) (λj.(λi.(j (j (j (j i)))))))
1: (((λa.(λb.(((λc.(λd.d)) a) b))) (λe.(λj.j))) (λx.(λy.(x (x (x (x y)))))))
2: (((λx.(λy.((λa.a) y))) (λb.(λc.c))) (λd.(λe.(d (d (d (d e)))))))
3: (((λx.(λy.y)) (λa.(λb.b))) (λc.(λd.(c (c (c (c d)))))))
4: ((λa.a) (λx.(λy.(x (x (x (x y)))))))
5: (λx.(λy.(x (x (x (x y))))))


reduction steps: 5
norm term:       (λx.(λy.(x (x (x (x y))))))


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [7]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

0: ((λd.(λe.(λj.(((d (λi.(λx.(x (i e))))) (λy.j)) (λa.a))))) (λb.(λc.c)))
1: (λx.(λy.((((λa.(λb.b)) (λc.(λd.(d (c x))))) (λe.y)) (λj.j))))
2: (λx.(λy.(((λa.a) (λb.y)) (λc.c))))
3: (λx.(λy.((λa.y) (λb.b))))
4: (λx.(λy.y))


reduction steps: 4
norm term:       (λx.(λy.y))


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [8]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

0: ((λx.(λy.(λa.(((x (λb.(λc.(c (b y))))) (λd.a)) (λe.e))))) (λj.(λi.(j (j (j (j (j i))))))))
1: (λx.(λy.((((λa.(λb.(a (a (a (a (a b))))))) (λc.(λd.(d (c x))))) (λe.y)) (λj.j))))
2: (λx.(λy.(((λa.((λb.(λc.(c (b x)))) ((λd.(λe.(e (d x)))) ((λj.(λi.(i (j x)))) ((λn.(λm.(m (n x)))) ((λt.(λr.(r (t x)))) a)))))) (λq.y)) (λw.w))))
3: (λx.(λy.(((λa.((λb.(λc.(c (b x)))) ((λd.(λe.(e (d x)))) ((λj.(λi.(i (j x)))) ((λn.(λm.(m (n x)))) (λt.(t (a x)))))))) (λr.y)) (λq.q))))
4: (λx.(λy.(((λa.((λb.(λc.(c (b x)))) ((λd.(λe.(e (d x)))) ((λj.(λi.(i (j x)))) (λn.(n ((λm.(m (a x))) x))))))) (λt.y)) (λr.r))))
5: (λj.(λi.(((λn.((λm.(λt.(t (m j)))) ((λx.(λy.(y (x j)))) ((λa.(λb.(b (a j)))) (λc.(c (j (n j)))))))) (λd.i)) (λe.e))))
6: (λx.(λy.(((λa.((λb.(λc.(c (b x)))) ((λd.(λe.(e (d x)))) (λj.(j ((λi.(i (x (a x)))) x)))))) (λn.y)) (λm.m))))
7: (λx.(λy.(((λa.((λb.(λc.(c (b x)))) ((λd.(λe.(e (d x)))) (λj.(j (x (x (a x)))))))) (λi.y)) (λn.n))))
8: (λd.(λe.(((λj.((λi.(λx.(x (i d)))) (λy.(y ((λa.(a (d (d (j d)))))

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [9]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

0: (((λx.(λy.((y (λa.(λb.(λc.(b ((a b) c)))))) x))) (λd.(λe.e))) (λj.(λi.i)))
1: ((λa.((a (λb.(λc.(λd.(c ((b c) d)))))) (λe.(λj.j)))) (λx.(λy.y)))
2: (((λx.(λy.y)) (λa.(λb.(λc.(b ((a b) c)))))) (λd.(λe.e)))
3: ((λx.x) (λy.(λa.a)))
4: (λx.(λy.y))


reduction steps: 4
norm term:       (λx.(λy.y))


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [10]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

0: (((λx.(λy.((y (λa.(λb.(λc.(b ((a b) c)))))) x))) (λd.(λe.(d (d e))))) (λj.(λi.(j (j (j i))))))
1: ((λx.((x (λy.(λa.(λb.(a ((y a) b)))))) (λc.(λd.(c (c d)))))) (λe.(λj.(e (e (e j))))))
2: (((λx.(λy.(x (x (x y))))) (λa.(λb.(λc.(b ((a b) c)))))) (λd.(λe.(d (d e)))))
3: ((λx.((λy.(λa.(λb.(a ((y a) b))))) ((λc.(λd.(λe.(d ((c d) e))))) ((λj.(λi.(λn.(i ((j i) n))))) x)))) (λm.(λt.(m (m t)))))
4: ((λe.((λj.(λi.(λn.(i ((j i) n))))) ((λm.(λx.(λy.(x ((m x) y))))) (λa.(λb.(a ((e a) b))))))) (λc.(λd.(c (c d)))))
5: ((λx.((λy.(λa.(λb.(a ((y a) b))))) (λc.(λd.(c (((λe.(λj.(e ((x e) j)))) c) d)))))) (λi.(λn.(i (i n)))))
6: ((λx.((λy.(λa.(λb.(a ((y a) b))))) (λc.(λd.(c ((λe.(c ((x c) e))) d)))))) (λj.(λi.(j (j i)))))
7: ((λy.((λa.(λb.(λc.(b ((a b) c))))) (λd.(λe.(d (d ((y d) e))))))) (λj.(λx.(j (j x)))))
8: ((λx.(λy.(λa.(y (((λb.(λc.(b (b ((x b) c))))) y) a))))) (λd.(λe.(d (d e)))))
9: ((λx.(λy.(λa.(y ((λb.(y (y ((x y) b)))) a))))) (λc.(λd.(c (c d)))))
10: ((λx.(λy.(λa.(y (y (y ((x y) a))))))) (λb.(

# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [11]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

0: (((λx.(λy.((y (λa.(λb.(λc.(((a (λd.(λe.(e (d b))))) (λj.c)) (λi.i)))))) x))) (λn.(λm.(n (n (n (n m))))))) (λt.(λr.(t r))))
1: ((λy.((y (λa.(λb.(λc.(((a (λd.(λe.(e (d b))))) (λj.c)) (λi.i)))))) (λn.(λm.(n (n (n (n m)))))))) (λt.(λx.(t x))))
2: (((λx.(λy.(x y))) (λa.(λb.(λc.(((a (λd.(λe.(e (d b))))) (λj.c)) (λi.i)))))) (λn.(λm.(n (n (n (n m)))))))
3: ((λx.((λy.(λa.(λb.(((y (λc.(λd.(d (c a))))) (λe.b)) (λj.j))))) x)) (λi.(λn.(i (i (i (i n)))))))
4: ((λx.(λy.(λa.(((x (λb.(λc.(c (b y))))) (λd.a)) (λe.e))))) (λj.(λi.(j (j (j (j i)))))))
5: (λj.(λx.((((λy.(λa.(y (y (y (y a)))))) (λb.(λc.(c (b j))))) (λd.x)) (λe.e))))
6: (λx.(λy.(((λa.((λb.(λc.(c (b x)))) ((λd.(λe.(e (d x)))) ((λj.(λi.(i (j x)))) ((λn.(λm.(m (n x)))) a))))) (λt.y)) (λr.r))))
7: (λx.(λy.(((λa.((λb.(λc.(c (b x)))) ((λd.(λe.(e (d x)))) ((λj.(λi.(i (j x)))) (λn.(n (a x))))))) (λm.y)) (λt.t))))
8: (λx.(λy.(((λa.((λb.(λc.(c (b x)))) ((λd.(λe.(e (d x)))) (λj.(j ((λi.(i (a x))) x)))))) (λn.y)) (λm.m))))
9: (λx.(λy.(((λa.((λb.(λc.(c

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9())

0: (((λx.(λy.((y (λa.(λb.(λc.(((a (λd.(λe.(e (d b))))) (λj.c)) (λi.i)))))) x))) (λn.(λm.(n (n (n m)))))) (λt.(λr.(t (t (t (t (t r))))))))
1: ((λe.((e (λj.(λi.(λn.(((j (λm.(λt.(t (m i))))) (λx.n)) (λy.y)))))) (λa.(λb.(a (a (a b))))))) (λc.(λd.(c (c (c (c (c d))))))))
2: (((λx.(λy.(x (x (x (x (x y))))))) (λa.(λb.(λc.(((a (λd.(λe.(e (d b))))) (λj.c)) (λi.i)))))) (λn.(λm.(n (n (n m))))))
3: ((λl.((λz.(λv.(λx_1.(((z (λy_1.(λa_1.(a_1 (y_1 v))))) (λb_1.x_1)) (λc_1.c_1))))) ((λd_1.(λe_1.(λj_1.(((d_1 (λi_1.(λn_1.(n_1 (i_1 e_1))))) (λm_1.j_1)) (λt_1.t_1))))) ((λx.(λy.(λa.(((x (λb.(λc.(c (b y))))) (λd.a)) (λe.e))))) ((λj.(λi.(λn.(((j (λm.(λt.(t (m i))))) (λr.n)) (λq.q))))) ((λw.(λu.(λo.(((w (λp.(λs.(s (p u))))) (λf.o)) (λg.g))))) l)))))) (λh.(λk.(h (h (h k))))))
4: ((λx.((λy.(λa.(λb.(((y (λc.(λd.(d (c a))))) (λe.b)) (λj.j))))) ((λi.(λn.(λm.(((i (λt.(λr.(r (t n))))) (λq.m)) (λw.w))))) ((λu.(λo.(λp.(((u (λs.(λf.(f (s o))))) (λg.p)) (λh.h))))) ((λk.(λl.(λz.(((k (λv.(λx_1.(x_1 (v l))))) (λy_1.z)) (λa

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [13]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

0: (((λx.(λy.((λa.((a (λb.(λc.(λd.d)))) (λe.(λj.e)))) (((λi.(λn.((n (λm.(λt.(λr.(((m (λq.(λw.(w (q t))))) (λu.r)) (λo.o)))))) i))) x) y)))) (λp.(λs.(p (p (p s)))))) (λf.(λg.(f (f g)))))
1: (((λu.(λo.((λp.((p (λs.(λf.(λx.x)))) (λy.(λa.y)))) ((λb.((b (λc.(λd.(λe.(((c (λj.(λi.(i (j d))))) (λn.e)) (λm.m)))))) u)) o)))) (λt.(λr.(t (t (t r)))))) (λq.(λw.(q (q w)))))
2: (((λx.(λy.((λa.((a (λb.(λc.(λd.d)))) (λe.(λj.e)))) ((y (λi.(λn.(λm.(((i (λt.(λr.(r (t n))))) (λq.m)) (λw.w)))))) x)))) (λu.(λo.(u (u (u o)))))) (λp.(λs.(p (p s)))))
3: (((λx.(λy.((((y (λa.(λb.(λc.(((a (λd.(λe.(e (d b))))) (λj.c)) (λi.i)))))) x) (λn.(λm.(λt.t)))) (λr.(λq.r))))) (λw.(λu.(w (w (w u)))))) (λo.(λp.(o (o p)))))
4: ((λd.((((d (λe.(λj.(λi.(((e (λn.(λm.(m (n j))))) (λt.i)) (λr.r)))))) (λq.(λw.(q (q (q w)))))) (λu.(λo.(λx.x)))) (λy.(λa.y)))) (λb.(λc.(b (b c)))))
5: (((((λx.(λy.(x (x y)))) (λa.(λb.(λc.(((a (λd.(λe.(e (d b))))) (λj.c)) (λi.i)))))) (λn.(λm.(n (n (n m)))))) (λt.(λr.(λq.q)))) (λw.(λu.w)))
6: ((((λx.((λy.(λa.

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [14]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11())

0: (((λx.(λy.((λa.((a (λb.(λc.(λd.d)))) (λe.(λj.e)))) (((λi.(λn.((n (λm.(λt.(λr.(((m (λq.(λw.(w (q t))))) (λu.r)) (λo.o)))))) i))) x) y)))) (λp.(λs.(p (p s))))) (λf.(λg.(f (f (f (f (f g))))))))
1: (((λo.(λp.((λs.((s (λf.(λx.(λy.y)))) (λa.(λb.a)))) ((λc.((c (λd.(λe.(λj.(((d (λi.(λn.(n (i e))))) (λm.j)) (λt.t)))))) o)) p)))) (λr.(λq.(r (r q))))) (λw.(λu.(w (w (w (w (w u))))))))
2: (((λx.(λy.((λa.((a (λb.(λc.(λd.d)))) (λe.(λj.e)))) ((y (λi.(λn.(λm.(((i (λt.(λr.(r (t n))))) (λq.m)) (λw.w)))))) x)))) (λu.(λo.(u (u o))))) (λp.(λs.(p (p (p (p (p s))))))))
3: (((λx.(λy.((((y (λa.(λb.(λc.(((a (λd.(λe.(e (d b))))) (λj.c)) (λi.i)))))) x) (λn.(λm.(λt.t)))) (λr.(λq.r))))) (λw.(λu.(w (w u))))) (λo.(λp.(o (o (o (o (o p))))))))
4: ((λe.((((e (λj.(λi.(λn.(((j (λm.(λt.(t (m i))))) (λr.n)) (λq.q)))))) (λw.(λu.(w (w u))))) (λo.(λx.(λy.y)))) (λa.(λb.a)))) (λc.(λd.(c (c (c (c (c d))))))))
5: (((((λx.(λy.(x (x (x (x (x y))))))) (λa.(λb.(λc.(((a (λd.(λe.(e (d b))))) (λj.c)) (λi.i)))))) (λn.(λm.(n (n m))))) (λ

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [15]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12(), steps_lim=100)

0: (((λi_1.(λn_1.(((λm_1.(λt_1.((((λr_1.(λq_1.(λw_1.((r_1 q_1) w_1)))) m_1) t_1) m_1))) (((λu_1.(λo_1.((λp_1.((p_1 (λs_1.(λx.(λy.y)))) (λa.(λb.a)))) (((λc.(λd.((d (λe.(λj.(λi.(((e (λn.(λm.(m (n j))))) (λt.i)) (λr.r)))))) c))) u_1) o_1)))) i_1) n_1)) (((λq.(λw.((λu.((u (λo.(λp.(λs.s)))) (λf.(λg.f)))) (((λh.(λk.((k (λl.(λz.(λv.(((l (λx_1.(λy_1.(y_1 (x_1 z))))) (λa_1.v)) (λb_1.b_1)))))) h))) q) w)))) n_1) i_1)))) (λc_1.(λd_1.(c_1 (c_1 d_1))))) (λe_1.(λj_1.(e_1 (e_1 j_1)))))
1: (((λx.(λy.(((λa.(λb.((((λc.(λd.(λe.((c d) e)))) a) b) a))) (((λj.(λi.((λn.((n (λm.(λt.(λr.r)))) (λq.(λw.q)))) (((λu.(λo.((o (λp.(λs.(λf.(((p (λg.(λh.(h (g s))))) (λk.f)) (λl.l)))))) u))) j) i)))) x) y)) (((λz.(λv.((λx_1.((x_1 (λy_1.(λa_1.(λb_1.b_1)))) (λc_1.(λd_1.c_1)))) ((λe_1.((e_1 (λj_1.(λi_1.(λn_1.(((j_1 (λm_1.(λt_1.(t_1 (m_1 i_1))))) (λr_1.n_1)) (λq_1.q_1)))))) z)) v)))) y) x)))) (λw_1.(λu_1.(w_1 (w_1 u_1))))) (λo_1.(λp_1.(o_1 (o_1 p_1)))))
2: (((λx.(λy.(((λa.(λb.((((λc.(λd.(λe.((c d) e)))) a) b) a))) (((λj.(λi

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [16]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13(), steps_lim=100)

0: (((λx.(λy.(((λa.(λb.((((λc.(λd.(λe.((c d) e)))) a) b) a))) (((λj.(λi.((λn.((n (λm.(λt.(λr.r)))) (λq.(λw.q)))) (((λu.(λo.((o (λp.(λs.(λf.(((p (λg.(λh.(h (g s))))) (λk.f)) (λl.l)))))) u))) j) i)))) x) y)) (((λz.(λv.((λx_1.((x_1 (λy_1.(λa_1.(λb_1.b_1)))) (λc_1.(λd_1.c_1)))) (((λe_1.(λj_1.((j_1 (λi_1.(λn_1.(λm_1.(((i_1 (λt_1.(λr_1.(r_1 (t_1 n_1))))) (λq_1.m_1)) (λw_1.w_1)))))) e_1))) z) v)))) y) x)))) (λu_1.(λo_1.(u_1 o_1)))) (λp_1.(λs_1.(p_1 (p_1 (p_1 (p_1 s_1)))))))
1: (((λj_1.(λi_1.(((λn_1.(λm_1.((((λt_1.(λr_1.(λq_1.((t_1 r_1) q_1)))) n_1) m_1) n_1))) (((λw_1.(λu_1.((λo_1.((o_1 (λp_1.(λx.(λy.y)))) (λa.(λb.a)))) (((λc.(λd.((d (λe.(λj.(λi.(((e (λn.(λm.(m (n j))))) (λt.i)) (λr.r)))))) c))) w_1) u_1)))) j_1) i_1)) (((λq.(λw.((λu.((u (λo.(λp.(λs.s)))) (λf.(λg.f)))) ((λh.((h (λk.(λl.(λz.(((k (λv.(λx_1.(x_1 (v l))))) (λy_1.z)) (λa_1.a_1)))))) q)) w)))) i_1) j_1)))) (λb_1.(λc_1.(b_1 c_1)))) (λd_1.(λe_1.(d_1 (d_1 (d_1 (d_1 e_1)))))))
2: (((λx.(λy.(((λa.(λb.((((λc.(λd.(λe.((c d) e)))) a) b) a)

In [17]:
from calculus_path_mod.json_serialization import *

inf_term = term_test_1()
dict_from_term = term_to_dict(inf_term)
term_from_dict = dict_to_term(dict_from_term)

In [18]:
print(dict_from_term)

{'kind': 'app', 'subj': {'kind': 'lambda', 'var': 0, 'body': {'kind': 'lambda', 'var': 1, 'body': {'kind': 'atom', 'var': 1}}}, 'obj': {'kind': 'app', 'subj': {'kind': 'lambda', 'var': 0, 'body': {'kind': 'app', 'subj': {'kind': 'atom', 'var': 0}, 'obj': {'kind': 'atom', 'var': 0}}}, 'obj': {'kind': 'lambda', 'var': 0, 'body': {'kind': 'app', 'subj': {'kind': 'atom', 'var': 0}, 'obj': {'kind': 'atom', 'var': 0}}}}}


In [19]:
print(term_from_dict.funky_str())

((λx.(λy.y)) ((λx.(x x)) (λx.(x x))))


In [20]:
print(term_from_dict)

((λ#9328. (λ#9329. #9329)) ((λ#9328. (#9328 #9328)) (λ#9328. (#9328 #9328))))


In [21]:
inf_term = term_test_1()._update_bound_vars()
dict_from_term = term_to_dict(inf_term)
term_from_dict = dict_to_term(dict_from_term)

print(dict_from_term)
print(term_from_dict)

{'kind': 'app', 'subj': {'kind': 'lambda', 'var': 0, 'body': {'kind': 'lambda', 'var': 1, 'body': {'kind': 'atom', 'var': 1}}}, 'obj': {'kind': 'app', 'subj': {'kind': 'lambda', 'var': 2, 'body': {'kind': 'app', 'subj': {'kind': 'atom', 'var': 2}, 'obj': {'kind': 'atom', 'var': 2}}}, 'obj': {'kind': 'lambda', 'var': 3, 'body': {'kind': 'app', 'subj': {'kind': 'atom', 'var': 3}, 'obj': {'kind': 'atom', 'var': 3}}}}}
((λ#9328. (λ#9329. #9329)) ((λ#9336. (#9336 #9336)) (λ#9337. (#9337 #9337))))


In [22]:
print(term_from_dict.funky_str())

((λx.(λy.y)) ((λa.(a a)) (λb.(b b))))


In [23]:
# list_terms = [
#     term_test_1(),
#     term_test_2(),
#     term_test_3(),
#     term_test_4(),
#     term_test_5()
# ]


# save_terms("saved_terms.dat", list_terms, is_overwrite=True)

In [24]:
list_terms = load_terms("saved_terms.dat")
for t in list_terms:
    print("*******")
    print(t.funky_str())
    print(t.normalize(LOStrategy())[0].funky_str())

FileNotFoundError: [Errno 2] No such file or directory: 'saved_terms.dat'

In [25]:
tttt = term_test_1()
strg = RandomOuterStrategy(prob_norm="sum")
strg.redex_index(tttt)

[1. 2.]
[2. 1.]
[0.66666667 0.33333333]


5

In [26]:
tttt = term_test_2()
strg = RandomOuterStrategy(prob_norm="sum")
strg.redex_index(tttt)

[3.]
[1.]
[1.]


3

In [27]:
tttt = term_test_13()
strg = RandomOuterStrategy(prob_norm="sum")
strg.redex_index(tttt)

[ 2.  6. 11.  8. 11. 13.  7. 10. 12.]
[12.  8.  3.  6.  3.  1.  7.  4.  2.]
[0.26086957 0.17391304 0.06521739 0.13043478 0.06521739 0.02173913
 0.15217391 0.08695652 0.04347826]


27

In [28]:
tttt = term_test_1()
strg = RandomInnerStrategy(prob_norm="softmax")
strg.redex_index(tttt)

5

In [29]:
tttt = term_test_2()
strg = RandomInnerStrategy(prob_norm="softmax")
strg.redex_index(tttt)

3

In [30]:
tttt = term_test_13()
strg = RandomInnerStrategy(prob_norm="softmax")
strg.redex_index(tttt)

40

# Compare different powers

In [33]:
tttt = term_test_13()
strg = RandomOuterStrategy(prob_norm="softmax")
strg.redex_index(tttt)

[ 2.  6. 11.  8. 11. 13.  7. 10. 12.]
[12.  8.  3.  6.  3.  1.  7.  4.  2.]
[9.72595448e-01 1.78137070e-02 1.20027814e-04 2.41082308e-03
 1.20027814e-04 1.62439982e-05 6.55329658e-03 3.26269425e-04
 4.41557650e-05]


2

In [34]:
tttt = term_test_13()
strg = RandomOuterStrategy(prob_norm="sum")
strg.redex_index(tttt)

[ 2.  6. 11.  8. 11. 13.  7. 10. 12.]
[12.  8.  3.  6.  3.  1.  7.  4.  2.]
[0.26086957 0.17391304 0.06521739 0.13043478 0.06521739 0.02173913
 0.15217391 0.08695652 0.04347826]


70

In [35]:
tttt = term_test_13()
strg = RandomOuterStrategy(prob_norm="pow_2")
strg.redex_index(tttt)

[ 2.  6. 11.  8. 11. 13.  7. 10. 12.]
[12.  8.  3.  6.  3.  1.  7.  4.  2.]
[0.43373494 0.19277108 0.02710843 0.10843373 0.02710843 0.00301205
 0.14759036 0.04819277 0.01204819]


70

In [36]:
tttt = term_test_13()
strg = RandomOuterStrategy(prob_norm="pow_3")
strg.redex_index(tttt)

[ 2.  6. 11.  8. 11. 13.  7. 10. 12.]
[12.  8.  3.  6.  3.  1.  7.  4.  2.]
[5.90567327e-01 1.74982912e-01 9.22761449e-03 7.38209159e-02
 9.22761449e-03 3.41763500e-04 1.17224880e-01 2.18728640e-02
 2.73410800e-03]


70

In [38]:
tttt = term_test_13()
strg = RandomOuterStrategy(prob_norm="pow_4")
strg.redex_index(tttt)

[ 2.  6. 11.  8. 11. 13.  7. 10. 12.]
[12.  8.  3.  6.  3.  1.  7.  4.  2.]
[7.15923215e-01 1.41416931e-01 2.79657506e-03 4.47452009e-02
 2.79657506e-03 3.45256180e-05 8.28960088e-02 8.83855821e-03
 5.52409888e-04]


2

In [39]:
tttt = term_test_13()
strg = RandomOuterStrategy(prob_norm="pow_5")
strg.redex_index(tttt)

[ 2.  6. 11.  8. 11. 13.  7. 10. 12.]
[12.  8.  3.  6.  3.  1.  7.  4.  2.]
[8.08615457e-01 1.06484340e-01 7.89663532e-04 2.52692330e-02
 7.89663532e-04 3.24964416e-06 5.46167695e-02 3.32763562e-03
 1.03988613e-04]


2